# Předzpracování datasetu TREC (coarse)

Tento notebook slouží k předzpracování datasetu TREC (coarse). Pro dataset jsou vytvořeny augmentované záznamy a předpočítány logity.
Nejprve jsou načteny všechny potřebné knihovny včetně vlastní sbírky objektů a funkcí.

In [1]:
from datasets import load_from_disk, Dataset, concatenate_datasets, ClassLabel, Features, Value, Sequence
from transformers import BertTokenizer, BertForSequenceClassification, BasicTokenizer
from torch.utils.data import  DataLoader
from tqdm.notebook import tqdm
import numpy as np
import os
import torch
import base
import copy

Ověření, že GPU je k dispozici a balíček torch je správně nakonfigurován.

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Konfigurace augmentačních parametrů.
Dataset se bude procházet desetkrát, každý token v záznamu bude na deset procent zamaskován a na třicet procent nahrazen jiným tokenem se stejným POS tagem. Po průchodu všech tokenů v záznamu může dojít s pravděpodobností dvacet procent ke zkrácení záznamu. 

In [3]:
augmentation_params = {"n_iter": 10, "p_mask":0.1, "p_pos": 0.3, "p_ng":0.2}

Získání datasetu, základního tokenizeru a jeho použití nad datasetem. 

In [4]:
tokenizer = BasicTokenizer(do_lower_case=True)
DATASET = "trec"

In [5]:
train_data = load_from_disk(f"~/data/{DATASET}/train_coarse")
sentences = list(map(lambda e: e["sentence"], train_data))

Pohled na průměrné délky záznamů dle počtu tokenů.

In [6]:
token_lengths = [len(tokenizer.tokenize(sentence)) for sentence in sentences]


In [7]:
sorted_token_lengths = sorted(token_lengths, reverse=True)
avg_tokens = np.mean(token_lengths)

In [8]:
print(sorted_token_lengths[0:25])
print(avg_tokens)
print(sorted_token_lengths[-25:])

[37, 36, 33, 33, 32, 31, 31, 31, 30, 30, 29, 29, 29, 29, 28, 28, 28, 27, 27, 27, 27, 27, 27, 27, 27]
10.821600550332493
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3]


Získání POS tagů jednotlivých tokenů v záznamech pro potřeby augmentace.

In [9]:
pos_tag_word_map_list = base.get_pos_tag_word_map(sentences, tokenizer=tokenizer)

In [10]:
print(train_data.features)

{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(names=['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM'], id=None)}


Spuštění procesu augmentace s popsanými parametry.

In [11]:
augmented_datasets = base.get_augmented_dataset(augmentation_params, train_data, pos_tag_word_map_list, tokenizer=tokenizer, include_idx=False)

Převedení nových záznamů do dataset objektu.

In [12]:
aug_datasets_formated = []
ds_schema = Features({
    "sentence": Value("string"),
    "label": ClassLabel(names=["ABBR", "ENTY", "DESC", "HUM", "LOC", "NUM"])
})

for iter in augmented_datasets:
    dataset = Dataset.from_dict(iter)
    dataset = dataset.cast(ds_schema)
    aug_datasets_formated.append(dataset)

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Ověření efektu augmentace, zde např. zamaskované slovo a změna slova.

In [ ]:
print(aug_datasets_formated[0][70])
print(train_data[70])

{'sentence': 'what body of water does [MASK] danube name flow into ?', 'label': 4}
{'sentence': 'What body of water does the Danube River flow into ?', 'label': 4}


Načtení učitelského modelu a jeho tokenizeru.

In [14]:
tokenizer = BertTokenizer.from_pretrained("carrassi-ni/bert-base-trec-question-classification")
model = BertForSequenceClassification.from_pretrained("carrassi-ni/bert-base-trec-question-classification", num_labels=6)
model.to(device)
model.eval()

torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/teacher_coarse.pth")

tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Předzpracování trénovací části datasetu a výpočet logitů učitelským modelem. Vypočtené logity se přidávají jako nový sloupec datasetu, naopak se odstraňují záznamy ponechané po tokenizeru učitele, které dále nejsou třeba. 

In [15]:
train_dataset = base.prepare_dataset(train_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
train_logits = base.generate_logits(train_dataloader, model)
train_dataset = train_dataset.add_column("logits", train_logits)

train_dataset = train_dataset.remove_columns(["token_type_ids", "attention_mask", "input_ids"])
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Tokenizing the provided dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
print(train_dataset[28])

{'labels': tensor(5), 'logits': tensor([-0.9769, -1.8649, -1.7079, -1.7614, -1.5043,  6.0615])}


Výpočet správnosti učitelských predikcí nad trénovací částí datasetu.

In [17]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/4361 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9867002980967668


Stejné kroky pro výpočet logitů jsou provedeny nad každou z deseti augmentovaných variant datasetu. Postupně je skrze záznamy iterováno. Dataset je nejprve tokenizován a následně je předán učitely k inferenci pro získání logitů, které jsou k datasetu uloženy. Odstraněny jsou poté nepotřebné pozůstatky tokenizace. V rámci průběhu zpracování rovnou dochází k filtraci augmentovaných záznamů dle popsaného mechanismu (notebook precompute_logits_10.ipynb).

Takto zpracované datasety se postupně spojují do jednoho celku. 

In [18]:
aug_clean_datasets = []
for dataset in tqdm(aug_datasets_formated, total=(len(aug_datasets_formated)), desc="Processing augmented datasets: "):
    aug_train_dataset = base.prepare_dataset(dataset, tokenizer)
    aug_train_dataloader = DataLoader(aug_train_dataset, batch_size=128, shuffle=False)
    aug_train_logits = base.generate_logits(aug_train_dataloader, model)
    aug_train_dataset = aug_train_dataset.add_column("logits", aug_train_logits)


    aug_train_dataset = aug_train_dataset.remove_columns(["token_type_ids", "attention_mask", "input_ids"])
    aug_train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

    print(base.check_acc(aug_train_dataset, "Accuracy for augmented dataset: "))

    aug_train_dataset = base.remove_diff_pred_class(train_dataset, aug_train_dataset, pytorch_dataset=False)
    
    print(base.check_acc(aug_train_dataset, "Accuracy for filtered dataset: "))

    aug_train_dataset.reset_format()
    aug_clean_datasets.extend(aug_train_dataset)

Processing augmented datasets:   0%|          | 0/10 [00:00<?, ?it/s]

Tokenizing the provided dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/35 [00:00<?, ?it/s]

Calculating accuracy based on the saved logits:   0%|          | 0/4361 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.7878926851639533


...

Calculating accuracy based on the saved logits:   0%|          | 0/4361 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.7952304517312543


Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Calculating accuracy based on the saved logits:   0%|          | 0/3481 [00:00<?, ?it/s]

Accuracy for filtered dataset:  0.990807239299052


In [ ]:
print(train_dataset.features)

{'sentence': Value(dtype='string', id=None), 'labels': ClassLabel(names=['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM'], id=None), 'logits': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}


Převedení spojeného a zpracovaného augmentovaného datasetu do dataset objektu.

In [20]:
ds_schema = Features({
    "sentence": Value("string"),
    "labels": ClassLabel(names=["ABBR", "ENTY", "DESC", "HUM", "LOC", "NUM"]),
    "logits": Sequence(feature=Value(dtype="float32")),
})

aug_dataset = Dataset.from_list(aug_clean_datasets)
aug_dataset = aug_dataset.cast(ds_schema)


Casting the dataset:   0%|          | 0/34561 [00:00<?, ? examples/s]

In [21]:
aug_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Výpočet správnosti nad zpracovanými datasety.

In [22]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))
print(base.check_acc(aug_dataset, "Accuracy for augmented dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/4361 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9867002980967668


Calculating accuracy based on the saved logits:   0%|          | 0/34561 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.989872978212436


Spojení původního a augmentovaného datasetu.

In [23]:
train_all_data = concatenate_datasets([train_dataset, aug_dataset])
train_all_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Získání správnosti nad touto kombinací.

In [24]:
print(base.check_acc(train_all_data, "Accuracy for combined dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/38922 [00:00<?, ?it/s]

Accuracy for combined dataset:  0.9895174965315245


In [ ]:
print(train_all_data.column_names)

['sentence', 'labels', 'logits']


In [ ]:
train_all_data.reset_format()

Uložení zpracovaných datasetů na disk.

In [27]:
train_all_data.save_to_disk(f"~/data/{DATASET}/train-logits-augmented_coarse")

Saving the dataset (0/1 shards):   0%|          | 0/38922 [00:00<?, ? examples/s]

In [28]:
train_dataset.reset_format()
train_dataset.save_to_disk(f"~/data/{DATASET}/train-logits_coarse")

Saving the dataset (0/1 shards):   0%|          | 0/4361 [00:00<?, ? examples/s]

Načtení zbylých částí datasetu (validační a testovací). Výpočet logitů je proveden pro každou z těchto částí stejným způsobem jako v příapdě trénovací části.

Nejprve je část tokenizována učitelem a vložena do dataloaderu.

In [29]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval_coarse")

eval_dataset = base.prepare_dataset(eval_data, tokenizer)
eval_dataloader = DataLoader(eval_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/1091 [00:00<?, ? examples/s]

In [30]:
test_data = load_from_disk(f"~/data/{DATASET}/test_coarse")

test_dataset = base.prepare_dataset(test_data, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Následně jsou pro každou část spočteny logity.

In [31]:
eval_logits = base.generate_logits(eval_dataloader, model)
test_logits = base.generate_logits(test_dataloader, model)

Generating logits for given dataset:   0%|          | 0/9 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/4 [00:00<?, ?it/s]

Logity jsou přidány jako nový sloupec, odstraněny jsou již nepotřebné pozůstatky tokenizace.

In [32]:
eval_dataset.reset_format()
eval_dataset = eval_dataset.add_column("logits", eval_logits)
eval_dataset = eval_dataset.remove_columns(["token_type_ids", "input_ids", "attention_mask"])

In [33]:
test_dataset.reset_format()
test_dataset = test_dataset.add_column("logits", test_logits)
test_dataset = test_dataset.remove_columns(["token_type_ids", "input_ids", "attention_mask"])

In [34]:
eval_dataset.save_to_disk(f"~/data/{DATASET}/eval-logits_coarse")
test_dataset.save_to_disk(f"~/data/{DATASET}/test-logits_coarse")

Saving the dataset (0/1 shards):   0%|          | 0/1091 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Uložení dat na disk a vypočtení správnosti predikcí.

In [35]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_coarse")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_coarse")

eval_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")
test_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

print(base.check_acc(eval_data, "Accuracy for base eval dataset: "))
print(base.check_acc(test_data, "Accuracy for base test dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/1091 [00:00<?, ?it/s]

Accuracy for base eval dataset:  0.9825847846012832


Calculating accuracy based on the saved logits:   0%|          | 0/500 [00:00<?, ?it/s]

Accuracy for base test dataset:  0.978


Výpočet výkonnostních metrik a velikosti u učitelského modelu.

In [36]:
train_dataset = base.prepare_dataset(train_data, tokenizer)

train_data_gpu = copy.deepcopy(train_dataset)
train_data_gpu.set_format(type="torch", columns=["input_ids", "attention_mask"], device="cuda")
gpu_data_loader = DataLoader(train_data_gpu, batch_size=1, shuffle=False)

train_data_cpu = copy.deepcopy(train_dataset)
train_data_cpu.set_format(type="torch", columns=["input_ids", "attention_mask"], device="cpu")
cpu_data_loader = DataLoader(train_data_cpu, batch_size=1, shuffle=False)

In [37]:
base.count_parameters(model)

model size: 413.196MB.
Total Trainable Params: 108314886.


,Modules,Parameters
0,bert.embeddings.word_embeddings.weight,22268928
1,bert.embeddings.position_embeddings.weight,393216
2,bert.embeddings.token_type_embeddings.weight,1536
3,bert.embeddings.LayerNorm.weight,768
4,bert.embeddings.LayerNorm.bias,768
...,...,...
196,bert.encoder.layer.11.output.LayerNorm.bias,768
197,bert.pooler.dense.weight,589824
198,bert.pooler.dense.bias,768
199,classifier.weight,4608


In [38]:
cpu_benchmark = base.BenchMarkRunner(model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  55.43 ms
  1 measurement, 1000 runs , 4 threads


In [39]:
gpu_benchmark = base.BenchMarkRunner(model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  6.26 ms
  1 measurement, 1000 runs , 4 threads


In [40]:
base.get_scores(test_data)

F1 score: 0.9804068214493165
Accuracy: 0.978
Precision: 0.9815953536084548
Recall: 0.9794343973270311
